In [28]:
%%capture
!pip install langchain==0.0.167
!pip install openai

In [44]:
from langchain.llms import OpenAI
from langchain.tools import BaseTool
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, Tool, tool

import warnings
warnings.filterwarnings('ignore')


class LCTAGI():
    def __init__(
        self,
        model_name = 'gpt-4'
        ):
        self.model_name = model_name

    
    def _generalize(self, input_prompt_):
        print("> Generalize the input task.")

        generalize_llm = OpenAI(temperature=0,model_name = self.model_name)

        prompt = """
        Please generalize the following sentences by replacing any numerical or other parts of the sentences with letters.
        Output is only the text after rewriting.

        ------------
        {input}
        ------------
        """.format(input = input_prompt_)

        responese = generalize_llm(prompt)

        print(f'Generalized Task：{responese}')

        return responese
        

    def _decide(self, input_prompt_, tools_):
        print("> Determine if you should make a tool.")
        
        decide_llm = OpenAI(temperature=0,model_name = self.model_name)

        description_list = ''
        for tool_ in tools_:
            description = tool_.description
            description_list += description + ','
        description_list = '[' + description_list + ']'

        prompt = """
        You are an agent that determines if the input task is executable. 
        All you can do is to be included in {exec_list}. 
        Answer "Yes." if you can perform the task, or "No." if you cannot.
        
        -----------
        The entered task is:{input}
        -----------
        """.format(exec_list = description_list, input = input_prompt_)

        responese = decide_llm(prompt)

        if responese == "Yes.":
            print('You do not need to create a tool to run it.')

        elif responese == "No.":
            print('You must create the tool before executing.')


        return responese


    def _tool_make(self, input_prompt_, folder_path_):
        print("> Create a TOOL.")

        tool_llm = OpenAI(temperature=0, model_name = self.model_name)


        create_prompt = """
        Create a python class that can execute {input} with a single string as input.
        Output should be code only.
        The following code was created with the input "multiply two numbers". Please create it like this code.
        Do not enclose the output in ``python ``` or the like.
        Do not change the class NewTool(BaseTool): in the code.
        
        ------------------
        class NewTool(BaseTool):
            name = "MultiplicationTool"
            description = "used for multiplication. The input is two numbers. For example, if you want to multiply 1 by 2, the input is '1,2'."

            def _run(self, query: str) -> str:
                "Use the tool."
                a, b = query.split(",")
                c = int(a) * int(b)
                result = c

            return result 

            async def _arun(self, query: str) -> str:
                "Use the tool asynchronously."
                raise NotImplementedError("BingSearchRun does not support async")
        ------------------
        """.format(input = input_prompt_)

        code = tool_llm(create_prompt)
        #print('Created Code：\n' + code)



        name_prompt = """
        From the following code, extract the part written in "" after the name =.
        The final output is only the content of the "".

        ------------------
        {code}
        ------------------
        """.format(code = code)

        name = tool_llm(name_prompt)
        print('Created tool name：' + name)

        # Save to File
        if folder_path_ != None:
            with open(folder_path_ + f'{name}.py', mode='w') as file:
                file.write(code)
        

        return code


    def _execute(self,input_prompt_, tools_):
        print("> Execute using the newly created tool.")

        excute_llm = OpenAI(temperature=0, model_name = self.model_name)

        agent = initialize_agent(tools_, excute_llm, agent="zero-shot-react-description", verbose=True)

        agent.run(input_prompt_)

        return


    def run(self, input_prompt_, tools_, folder_path_ = None):

        generalized_task = self._generalize(input_prompt_)
    
        output = self._decide(generalized_task, tools_)

        if output == "Yes.":
            self._execute(input_prompt_, tools_)

        elif output == "No.":
            new_tool_code = self._tool_make(generalized_task, folder_path_)

            exec(new_tool_code, globals())
            
            new_tool = NewTool()           
            tools_.append(new_tool)
            
            self._execute(input_prompt_, tools_)
        
        return 

In [45]:
input = 'Sum of prime numbers from 1 to 50'

folder_path = '/home/langchain-tools/data/'
tools = []

lctagi = LCTAGI()

lctagi.run(input, tools, folder_path)

#lctagi.run(input, tools)

> Generalize the input task.
Generalized Task：Sum of prime numbers from A to B
> Determine if you should make a tool.
It is necessary to create a tool, so run it after creating the tool.
> Create a TOOL.
Created tool name：PrimeSumTool
[NewTool(name='PrimeSumTool', description="used for calculating the sum of prime numbers between A and B. The input is a single string containing two numbers separated by a comma. For example, if you want to find the sum of prime numbers between 2 and 10, the input is '2,10'.", args_schema=None, return_direct=False, verbose=False, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0xffff8c0b6c50>)]
> Execute using the newly created tool.


> Entering new AgentExecutor chain...
I need to find the sum of prime numbers between 1 and 50. I can use the PrimeSumTool for this.
Action: PrimeSumTool
Action Input: 1,50
Observation: 328
Thought:Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_48659/4038601008.py", line 8, in <module>
    lctagi.run(input, tools, folder_path)
  File "/tmp/ipykernel_48659/1125205153.py", line 164, in run
    self._execute(input_prompt_, tools_)
  File "/tmp/ipykernel_48659/1125205153.py", line 137, in _execute
    agent.run(input_prompt_)
  File "/opt/conda/lib/python3.10/site-packages/langchain/chains/base.py", line 213, in run
    if self.memory is not None:
  File "/opt/conda/lib/python3.10/site-packages/langchain/chains/base.py", line 116, in __call__
    only one param.
  File "/opt/conda/lib/python3.10/site-packages/langchain/chains/base.py", line 113, in __call__
  File "/opt/conda/lib/python3.10/site-packages/langchain/agents/agent.py", line 792, in _call
    observation = InvalidTool().run(
  File "/opt/conda/lib/python